In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
rna = pd.read_csv('../use_data/rna_data.csv', index_col = 0)
#relevant_genes = np.array(pd.read_csv('../use_data/relevant_genes.csv'))
#rna = rna.iloc[:,np.isin(rna.columns, relevant_genes)]
            
rna['cell_line'] = rna.index
rna

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1,cell_line
ACH-001113,4.331992,0.000000,7.364660,2.792855,4.471187,0.028569,1.226509,3.044394,6.500005,4.739848,...,0.189034,0.201634,2.130931,0.555816,0.000000,0.275007,0.0,0.000000,0.000000,ACH-001113
ACH-001289,4.567424,0.584963,7.106641,2.543496,3.504620,0.000000,0.189034,3.813525,4.221877,3.481557,...,1.049631,0.321928,1.464668,0.632268,0.000000,0.014355,0.0,0.000000,0.000000,ACH-001289
ACH-001339,3.150560,0.000000,7.379118,2.333424,4.228049,0.056584,1.310340,6.687201,3.682573,3.273516,...,1.097611,0.831877,2.946731,0.475085,0.000000,0.084064,0.0,0.000000,0.042644,ACH-001339
ACH-001538,5.085340,0.000000,7.154211,2.545968,3.084064,0.000000,5.868390,6.165309,4.489928,3.956986,...,0.632268,0.298658,1.641546,0.443607,0.000000,0.028569,0.0,0.000000,0.000000,ACH-001538
ACH-000242,6.729417,0.000000,6.537917,2.456806,3.867896,0.799087,7.208478,5.570159,7.127117,4.568032,...,2.358959,0.084064,1.910733,0.000000,0.000000,0.464668,0.0,0.000000,0.000000,ACH-000242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000285,0.056584,0.000000,6.604368,3.266037,4.973152,0.411426,0.097611,0.704872,4.829850,5.178715,...,0.084064,1.310340,3.039138,0.344828,0.000000,0.000000,0.0,0.475085,0.042644,ACH-000285
ACH-002669,3.111031,0.000000,7.031329,1.541019,3.664483,0.014355,3.624101,6.805421,4.472488,4.397118,...,0.400538,0.356144,1.327687,0.000000,0.000000,0.014355,0.0,0.000000,0.000000,ACH-002669
ACH-001858,4.390943,0.000000,7.013239,1.887525,3.252476,0.028569,3.286881,6.902194,5.410748,3.401903,...,0.400538,0.613532,1.992768,0.704872,0.000000,1.464668,0.0,0.000000,0.526069,ACH-001858
ACH-001997,5.057450,0.000000,7.815191,2.538538,3.893362,0.028569,4.079805,6.971659,4.469886,3.463361,...,0.847997,1.292782,2.153805,0.687061,0.000000,0.000000,0.0,0.000000,0.000000,ACH-001997


In [3]:
fold = 3

In [14]:


train_test_names = pd.read_csv('../results/train_test_splits/train_test_names' +str(fold) +'.csv', index_col=0)
train_names = train_test_names[train_test_names['train_test']=='train']['cell_line']
test_names = train_test_names[train_test_names['train_test']=='test']['cell_line']

train_rna = rna[np.isin(rna.index, train_names)]
test_rna = rna[np.isin(rna.index, test_names)]

In [5]:
experiments = pd.read_csv('../use_data/prediction_targets.csv', index_col = 0)
train_experiments = experiments[np.isin(experiments['cell_line'], train_names)]
test_experiments = experiments[np.isin(experiments['cell_line'], test_names)]
train_experiments
print(test_experiments.shape,train_experiments.shape)

(109976, 8) (439576, 8)


In [6]:

drugs = experiments['DRUG'].drop_duplicates()
drugs

0          CYTARABINE
408        EPINASTINE
820       FLOXURIDINE
1289       VALRUBICIN
1759        ADAPALENE
             ...     
631598    PARDOPRUNOX
632984      RUBITECAN
634415         XL-647
635843      VERUBULIN
636320         VE-822
Name: DRUG, Length: 1135, dtype: object

In [7]:
experiments

,cell_line,DRUG,auc,logauc,auc_per_drug,means,stds,range
0,ACH-000879,CYTARABINE,1.677789,0.517477,2.367402,0.228684,0.121987,0.782691
1,ACH-000320,CYTARABINE,1.240300,0.215353,-0.109281,0.228684,0.121987,0.782691
2,ACH-001145,CYTARABINE,1.472333,0.386848,1.296560,0.228684,0.121987,0.782691
3,ACH-000873,CYTARABINE,1.207160,0.188270,-0.331294,0.228684,0.121987,0.782691
4,ACH-000855,CYTARABINE,1.229332,0.206471,-0.182097,0.228684,0.121987,0.782691
...,...,...,...,...,...,...,...,...
700916,ACH-001321,BREQUINAR,0.987409,-0.012671,0.979322,-0.184222,0.175173,1.078361
700917,ACH-001321,AZD8931,1.000000,0.000000,0.879937,-0.199376,0.226580,2.297688
700918,ACH-001321,AZD2014,1.000000,0.000000,1.643680,-0.279217,0.169873,1.025072
700919,ACH-001321,MOTESANIB,0.888088,-0.118685,-0.367226,-0.084827,0.092199,0.957449


In [8]:
rna_data_sanger = pd.read_csv('../results/data/sanger_RNA.csv',index_col=0) 
rna_data_sanger

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
cell_line,,,,,,,,,,,,,,,,,,,,,
ACH-000001,5.183487,0.000000,7.497612,2.107688,4.217231,0.042644,0.903038,5.722193,4.676944,3.720278,...,1.137504,0.000000,0.189034,1.794936,0.201634,0.000000,0.028569,0.000000,0.214125,0.014355
ACH-000002,0.176323,0.000000,5.702103,1.238787,3.119356,4.141596,0.163499,4.134221,4.111866,2.347666,...,1.952334,0.238787,0.150560,1.516015,0.000000,0.000000,0.028569,0.000000,0.000000,0.000000
ACH-000004,2.176323,0.000000,5.454505,2.480265,3.921246,0.887525,4.958843,3.949535,4.877253,4.829850,...,0.536053,0.505891,0.731183,2.643856,0.097611,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-000006,0.097611,0.000000,6.285032,2.763412,4.192194,3.858976,3.921246,5.300124,4.835419,4.955592,...,0.831877,0.097611,0.575312,2.563158,1.007196,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-000009,5.761019,0.111031,8.081190,2.241840,4.448901,0.000000,0.042644,7.130416,5.691255,4.432291,...,0.963474,0.000000,0.321928,2.319040,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-001603,3.152183,0.000000,6.033423,1.871844,2.664483,0.000000,0.014355,1.389567,4.053980,3.992768,...,0.855990,0.632268,0.464668,1.847997,1.084064,0.000000,0.000000,0.000000,0.000000,0.056584
ACH-000353,4.498251,0.000000,6.953149,2.060047,4.116864,0.124328,4.761817,6.214513,5.329124,3.463361,...,1.257011,0.782409,0.678072,2.488001,0.298658,0.014355,0.321928,0.014355,0.014355,0.000000
ACH-000725,3.049631,0.000000,6.812241,3.669027,4.338424,0.056584,0.275007,4.541019,5.300490,4.538538,...,3.364572,1.510962,0.594549,1.847997,0.042644,0.097611,0.014355,0.000000,0.097611,0.000000


In [9]:
rna_data_sanger.mean(axis=0)
train_rna.iloc[:,1:-9].mean(axis=0)

TNMD          0.073307
DPM1          6.503655
SCYL3         2.380713
C1orf112      3.649538
FGR           0.451868
                ...   
NOTCH2NLC     2.516613
SPDYE11       0.034549
AP001453.3    2.791648
H3C2          1.045814
H3C3          0.763170
Length: 19184, dtype: float64

In [10]:
sanger_results = pd.read_csv('../results/data/sanger_results.csv',index_col =0) 
sanger_results

,cell_line,drug,auc,auc_per_drug
0,ACH-000001,CAMPTOTHECIN,0.967187,1.219220
1,ACH-000002,CAMPTOTHECIN,0.711866,-1.322325
2,ACH-000004,CAMPTOTHECIN,0.742350,-1.018876
3,ACH-000006,CAMPTOTHECIN,0.752250,-0.920331
6,ACH-000009,CAMPTOTHECIN,0.948028,1.028506
...,...,...,...,...
137927,ACH-000876,JQ1-(+),0.900201,0.486184
137944,ACH-000902,JQ1-(+),0.595288,-2.043808
137948,ACH-000910,JQ1-(+),0.523851,-2.636547
137958,ACH-000930,JQ1-(+),0.830671,-0.090733


In [11]:
def crossval(drug, train_experiments, test_experiments, train_rna, test_rna, sanger_experiments, sanger_rna):
    train_experiments_now = train_experiments[train_experiments['DRUG']==drug]
    test_experiments_now = test_experiments[test_experiments['DRUG']==drug]

    train_data_and_labels = train_rna.merge(train_experiments_now, left_index=True, right_on = 'cell_line', how='inner')
    train_data = train_data_and_labels.iloc[:,1:-9]  
    train_label = train_data_and_labels['auc_per_drug']
    test_data_and_labels = test_rna.merge(test_experiments_now, left_index=True, right_on = 'cell_line', how='inner')
    cell_line = test_data_and_labels['cell_line']

    test_data = test_data_and_labels.iloc[:,1:-9]  
    print(train_data.shape, test_data.shape)
    test_label = test_data_and_labels['auc_per_drug']
    
    #scaling
    scaler = StandardScaler().fit(train_data)
    train_data = pd.DataFrame(scaler.transform(train_data), index = train_data.index, columns = train_data.columns)
    test_data = pd.DataFrame(scaler.transform(test_data), index = test_data.index, columns = test_data.columns)
    
    

    
    
    lasso = Lasso(alpha=0.1, max_iter=4000)    
    
    ridge = Ridge(alpha=0.1)
    EN = ElasticNet(alpha=0.1)
    #linear = LinearRegression()
    svr = SVR(kernel='rbf')
    #kernel_ridge = KernelRidge(alpha=0.1)

    
    lasso.fit(X = train_data, y = train_label)
    prediction_lasso = lasso.predict(test_data)
    
    ridge.fit(X = train_data, y = train_label)
    prediction_ridge = ridge.predict(test_data)
    
    #kernel_ridge.fit(X = train_data, y = train_label)
    #prediction_kernel_ridge = kernel_ridge.predict(test_data)
    
    EN.fit(X = train_data, y = train_label)
    prediction_EN = EN.predict(test_data)
    
    svr.fit(X = train_data, y = train_label)
    prediction_svr = svr.predict(test_data)
    
    prism_results = pd.DataFrame({'label': test_label, 
                         'lasso': prediction_lasso, 

                         'ridge': prediction_ridge, 
                         #'kernel_ridge': prediction_kernel_ridge, 

                         'elastic': prediction_EN, 
                         'svr': prediction_svr, 
                         'drugs':drug, 
                         'fold': fold, 
                         'cells': cell_line})
    
    ############################################################
    #sanger
    sanger_experiments_now = sanger_experiments[sanger_experiments['drug']==drug]
    if sanger_experiments_now.shape[0]==0:
        return prism_results, None
    
    sanger_data_and_labels = sanger_rna.merge(sanger_experiments_now, left_index = True, right_on = 'cell_line')
    
    sanger_data = sanger_data_and_labels.iloc[:,:-4]
    
    sanger_data = pd.DataFrame(scaler.transform(sanger_data), index = sanger_data.index, columns = sanger_data.columns)

    sanger_label = sanger_data_and_labels['auc_per_drug']
    sanger_cell_line = sanger_data_and_labels['cell_line']
    
    sanger_results = pd.DataFrame({
        'label': sanger_label,
        'lasso': lasso.predict(sanger_data),
        'ridge': ridge.predict(sanger_data),
        'elastic': EN.predict(sanger_data),
         'svr': svr.predict(sanger_data), 
         'drugs':drug, 
         'fold': fold, 
         'cells': sanger_cell_line  
    })
    
    ############################################################
    
    
    return prism_results, sanger_results


In [12]:
crossval(np.array(drugs)[1], train_experiments=train_experiments, test_experiments=test_experiments, train_rna=train_rna, test_rna=test_rna, sanger_experiments = sanger_results, sanger_rna = rna_data_sanger)

(328, 19193) (75, 19193)


(        label     lasso     ridge   elastic       svr       drugs  fold   
 809 -0.573579 -0.171145 -0.493400 -0.318953 -0.207509  EPINASTINE     3  \
 689 -0.768499  0.070813  0.400620  0.243687  0.012590  EPINASTINE     3   
 517  0.272595 -0.178061 -0.106966  0.052020  0.037712  EPINASTINE     3   
 438  0.273639  0.086343  0.316623  0.365070  0.128841  EPINASTINE     3   
 770 -1.301676 -0.075074  0.368867 -0.007778 -0.075100  EPINASTINE     3   
 ..        ...       ...       ...       ...       ...         ...   ...   
 719  1.067594 -0.122521 -0.320817 -0.332559 -0.158083  EPINASTINE     3   
 586 -2.119755  0.397589  0.394281  0.242531  0.058907  EPINASTINE     3   
 445  0.888465  0.138211  0.250723  0.324390  0.139044  EPINASTINE     3   
 552  0.522617 -0.094374  0.049899 -0.296028 -0.254162  EPINASTINE     3   
 473  4.907328 -0.496527 -0.098386 -0.741785 -0.194371  EPINASTINE     3   
 
           cells  
 809  ACH-000123  
 689  ACH-000313  
 517  ACH-000823  
 438  ACH-

In [13]:
#all_tests = [crossval(drug, train_experiments=train_experiments, test_experiments=test_experiments, train_rna=train_rna, test_rna=test_rna ) for drug in tqdm(np.array(drugs))]

In [14]:
all_tests = Parallel(n_jobs=10)(delayed(crossval)(drug,train_experiments, test_experiments, train_rna, test_rna, sanger_experiments = sanger_results, sanger_rna = rna_data_sanger) for drug in tqdm(np.array(drugs)[:]))


  1%|▎                                        | 10/1135 [00:00<00:30, 36.79it/s]/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy vers

  6%|██▌                                      | 70/1135 [00:39<11:51,  1.50it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 13%|█████▎                                  | 150/1135 [01:37<13:02,  1.26it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 37%|██████████████▊                         | 420/1135 [04:28<08:29,  1.40it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 42%|████████████████▉                       | 480/1135 [05:10<07:27,  1.46it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 43%|█████████████████▎                      | 490/1135 [05:16<07:01,  1.53it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered 

...

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret 

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
 82%|████████████████████████████████▊       | 930/1135 [09:53<01:42,  2.01it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret 

 93%|████████████████████████████████████▍  | 1060/1135 [11:17<00:49,  1.53it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

 99%|██████████████████████████████████████▍| 1120/1135 [11:55<00:09,  1.58it/s]/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/pkeyl/.local/lib/python3.10/site-packages/sklearn/

In [15]:
all_tests_combined_prism = pd.concat([tup[0] for tup in all_tests], axis=0)
all_tests_combined_sanger = pd.concat([tup[1] for tup in all_tests], axis=0)

all_tests_combined_prism

,label,lasso,ridge,elastic,svr,drugs,fold,cells
396,-0.468770,-0.157310,0.286041,-0.332315,-0.032570,CYTARABINE,3,ACH-000123
110,-0.812435,0.116754,0.250239,0.148214,-0.009549,CYTARABINE,3,ACH-000823
30,-1.183152,-0.055809,-0.186321,0.022498,0.053594,CYTARABINE,3,ACH-000138
355,-0.790371,0.110752,-0.020869,0.214591,-0.214298,CYTARABINE,3,ACH-000176
106,0.410190,-0.629569,-0.075382,-0.868271,-0.152470,CYTARABINE,3,ACH-000785
...,...,...,...,...,...,...,...,...
636481,-0.738488,-0.104339,-0.236547,-0.289905,-0.080005,VE-822,3,ACH-000542
636541,2.141372,0.286199,1.261148,0.531079,0.280218,VE-822,3,ACH-000217
636686,1.220122,-0.188380,-0.135299,0.002406,0.029166,VE-822,3,ACH-000505
636391,-2.703661,-0.034259,0.400691,0.155976,0.179537,VE-822,3,ACH-001128


In [16]:
all_tests_combined_sanger.to_csv('../results/other_models/sanger'+str(fold)+'.csv')
all_tests_combined_prism.to_csv('../results/other_models/prism'+str(fold)+'.csv')


(365, 19193) (93, 19193)
(319, 19193) (81, 19193)
(323, 19193) (86, 19193)
(356, 19193) (87, 19193)
(321, 19193) (88, 19193)
(340, 19193) (89, 19193)
(366, 19193) (93, 19193)
(315, 19193) (75, 19193)
(358, 19193) (88, 19193)
(714, 19193) (177, 19193)
(324, 19193) (86, 19193)
(343, 19193) (80, 19193)
(328, 19193) (72, 19193)
(678, 19193) (165, 19193)
(723, 19193) (183, 19193)
(714, 19193) (173, 19193)
(357, 19193) (91, 19193)
(364, 19193) (93, 19193)
(293, 19193) (77, 19193)
(319, 19193) (83, 19193)
(314, 19193) (80, 19193)
(309, 19193) (75, 19193)
(303, 19193) (77, 19193)
(316, 19193) (78, 19193)
(291, 19193) (77, 19193)
(317, 19193) (81, 19193)
(314, 19193) (75, 19193)
(253, 19193) (69, 19193)
(271, 19193) (75, 19193)
(265, 19193) (72, 19193)
(259, 19193) (52, 19193)
(275, 19193) (59, 19193)
(267, 19193) (59, 19193)
(23, 19193) (4, 19193)
(292, 19193) (75, 19193)
(673, 19193) (170, 19193)
(548, 19193) (136, 19193)
(278, 19193) (67, 19193)
(180, 19193) (47, 19193)
(633, 19193) (156, 19

(679, 19193) (170, 19193)
(305, 19193) (78, 19193)
(334, 19193) (83, 19193)
(337, 19193) (88, 19193)
(349, 19193) (88, 19193)
(335, 19193) (82, 19193)
(365, 19193) (91, 19193)
(326, 19193) (88, 19193)
(351, 19193) (84, 19193)
(369, 19193) (93, 19193)
(359, 19193) (91, 19193)
(353, 19193) (87, 19193)
(659, 19193) (152, 19193)
(712, 19193) (182, 19193)
(356, 19193) (91, 19193)
(368, 19193) (93, 19193)
(332, 19193) (85, 19193)
(721, 19193) (177, 19193)
(303, 19193) (73, 19193)
(718, 19193) (176, 19193)
(308, 19193) (80, 19193)
(314, 19193) (80, 19193)
(324, 19193) (82, 19193)
(316, 19193) (77, 19193)
(247, 19193) (59, 19193)
(273, 19193) (76, 19193)
(229, 19193) (61, 19193)
(296, 19193) (79, 19193)
(125, 19193) (33, 19193)
(630, 19193) (159, 19193)
(251, 19193) (51, 19193)
(543, 19193) (122, 19193)
(317, 19193) (82, 19193)
(566, 19193) (136, 19193)
(197, 19193) (46, 19193)
(291, 19193) (75, 19193)
(278, 19193) (74, 19193)
(68, 19193) (17, 19193)
(653, 19193) (166, 19193)
(331, 19193) (89,

(318, 19193) (86, 19193)
(345, 19193) (82, 19193)
(329, 19193) (85, 19193)
(681, 19193) (170, 19193)
(348, 19193) (93, 19193)
(703, 19193) (176, 19193)
(305, 19193) (77, 19193)
(356, 19193) (91, 19193)
(358, 19193) (88, 19193)
(346, 19193) (88, 19193)
(361, 19193) (89, 19193)
(361, 19193) (90, 19193)
(345, 19193) (92, 19193)
(367, 19193) (93, 19193)
(371, 19193) (94, 19193)
(371, 19193) (94, 19193)
(366, 19193) (93, 19193)
(365, 19193) (93, 19193)
(347, 19193) (87, 19193)
(366, 19193) (93, 19193)
(347, 19193) (91, 19193)
(289, 19193) (74, 19193)
(302, 19193) (81, 19193)
(325, 19193) (83, 19193)
(308, 19193) (80, 19193)
(333, 19193) (84, 19193)
(690, 19193) (173, 19193)
(325, 19193) (83, 19193)
(300, 19193) (79, 19193)
(279, 19193) (75, 19193)
(271, 19193) (75, 19193)
(265, 19193) (58, 19193)
(251, 19193) (55, 19193)
(227, 19193) (61, 19193)
(316, 19193) (81, 19193)
(309, 19193) (78, 19193)
(247, 19193) (58, 19193)
(664, 19193) (164, 19193)
(659, 19193) (166, 19193)
(330, 19193) (84, 19

(324, 19193) (78, 19193)
(366, 19193) (93, 19193)
(273, 19193) (68, 19193)
(729, 19193) (185, 19193)
(332, 19193) (82, 19193)
(660, 19193) (171, 19193)
(332, 19193) (81, 19193)
(349, 19193) (90, 19193)
(359, 19193) (90, 19193)
(667, 19193) (166, 19193)
(149, 19193) (40, 19193)
(700, 19193) (177, 19193)
(364, 19193) (94, 19193)
(344, 19193) (91, 19193)
(359, 19193) (92, 19193)
(720, 19193) (181, 19193)
(317, 19193) (83, 19193)
(302, 19193) (79, 19193)
(318, 19193) (80, 19193)
(676, 19193) (163, 19193)
(321, 19193) (83, 19193)
(287, 19193) (75, 19193)
(283, 19193) (75, 19193)
(294, 19193) (75, 19193)
(257, 19193) (56, 19193)
(260, 19193) (57, 19193)
(266, 19193) (58, 19193)
(290, 19193) (73, 19193)
(290, 19193) (76, 19193)
(640, 19193) (162, 19193)
(195, 19193) (50, 19193)
(290, 19193) (75, 19193)
(620, 19193) (151, 19193)
(338, 19193) (88, 19193)
(352, 19193) (91, 19193)
(352, 19193) (91, 19193)
(352, 19193) (89, 19193)
(218, 19193) (62, 19193)
(706, 19193) (171, 19193)
(277, 19193) (66